# 04b. Set Staging Alias on Model

**Purpose**: Set 'staging' alias on a registered model version for testing

**Prerequisites**:
* Model registered in Unity Catalog (04a_model_registration.ipynb)

**Outputs**:
* Model version has 'staging' alias
* Ready for testing and validation

In [0]:
# Install dependencies
%pip install --upgrade typing_extensions>=4.6.0 pydantic>=2.0.0 --quiet
!pip install -r /Workspace/Users/ashish.kamboj@tigeranalytics.com/home-credit-hyperpersonalization/requirements.txt

dbutils.library.restartPython()

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import sys
import os
from mlflow.tracking import MlflowClient

# Add project root to path
project_root = os.path.dirname(os.getcwd()) if os.getcwd().endswith('notebooks') else os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils.common_utils import load_config, setup_logging, print_section_header

print("✅ Imports successful")

✅ Imports successful


In [0]:
config = load_config('../config/config.yaml')
setup_logging(config)

print_section_header("Set Staging Alias")

model_name = config['mlflow']['databricks']['registered_model_name']
print(f"Model: {model_name}")


                               Set Staging Alias                                

Model: datafabric_catalog.customer_hc_silver.next_best_product_model


In [0]:
# Show current model versions and aliases
client = MlflowClient()

print("Current Model Versions:")
print("=" * 80)

all_versions = client.search_model_versions(f"name='{model_name}'")
for v in sorted(all_versions, key=lambda x: int(x.version)):
    version_details = client.get_model_version(model_name, v.version)
    aliases = version_details.aliases if hasattr(version_details, 'aliases') else []
    print(f"Version {v.version}: Aliases={aliases}, Status={v.status}")
    if v.description:
        print(f"  Description: {v.description[:80]}")

print("\n" + "=" * 80)

Current Model Versions:
Version 1: Aliases=[], Status=READY
  Description: Trained with current feature set & includes Top-K metrics | Run: 5b44f2102cbe477
Version 2: Aliases=[], Status=READY
  Description: Trained with current feature set & includes Top-K metrics | Run: 7cd08873f59543a
Version 3: Aliases=['production'], Status=READY
  Description: Trained with current feature set & includes Top-K metrics | Run: 9ba7b74e400a47a
Version 4: Aliases=['staging'], Status=READY
  Description: Trained with current feature set & includes Top-K metrics | Run: d8320cb03a64404
Version 5: Aliases=[], Status=READY
  Description: Trained with current feature set & includes Top-K metrics | Run: b5b7dcadd50b4c2



In [0]:
# Set 'staging' alias on the latest version
# You can change version_to_stage to any version number

# Get latest version
latest_version = max([int(v.version) for v in all_versions])
version_to_stage = latest_version  # Change this if you want to stage a different version

print(f"Setting 'staging' alias on Version {version_to_stage}...")

client.set_registered_model_alias(model_name, 'staging', str(version_to_stage))

print(f"\n✅ Version {version_to_stage} now has 'staging' alias!")

Setting 'staging' alias on Version 5...

✅ Version 5 now has 'staging' alias!


In [0]:
# Verify the alias was set
print("\nVerifying 'staging' alias...")

try:
    staging_version = client.get_model_version_by_alias(model_name, 'staging')
    print(f"✅ 'staging' alias is set on Version {staging_version.version}")
    print(f"\nModel URI: models:/{model_name}@staging")
    print(f"\nYou can now test this model using:")
    print(f"  mlflow.sklearn.load_model('models:/{model_name}@staging')")
except Exception as e:
    print(f"❌ Error: {str(e)}")


Verifying 'staging' alias...
✅ 'staging' alias is set on Version 5

Model URI: models:/datafabric_catalog.customer_hc_silver.next_best_product_model@staging

You can now test this model using:
  mlflow.sklearn.load_model('models:/datafabric_catalog.customer_hc_silver.next_best_product_model@staging')


In [0]:
print_section_header("Staging Alias Summary")

print(f"""
✅ Staging Alias Set Successfully!

Model: {model_name}
Version: {version_to_stage}
Alias: staging
Model URI: models:/{model_name}@staging

👉 Next Steps:
1. Test the model in staging environment
2. Run validation tests
3. When ready, run 04c_set_production_alias.ipynb to promote to production
""")


                             Staging Alias Summary                              


✅ Staging Alias Set Successfully!

Model: datafabric_catalog.customer_hc_silver.next_best_product_model
Version: 5
Alias: staging
Model URI: models:/datafabric_catalog.customer_hc_silver.next_best_product_model@staging

👉 Next Steps:
1. Test the model in staging environment
2. Run validation tests
3. When ready, run 04c_set_production_alias.ipynb to promote to production

